This notebook represents a first attempt at recreating the acceleration vector field used to propegate an atom through an Electric Field. The below code is based upon Mathematica code created by Anne Goodsell.

FFW | 10/16/2023

In [ ]:
#Upload basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
from scipy import constants

#### Loading in Stark Effect Data:

In [ ]:
#Determing server directory
current_directory = os.getcwd()
#print(current_directory)

In [ ]:
#Loading in our test CSV
fpath0 = current_directory
file = '/Constant_U.csv'

# Indicate sublevel just for consistency in other cells
sublevels = ['TEST']

starkmap_Vcm = {}
starkmap_Vm = {}

for sublevel in sublevels:
    filename = fpath0 + file
    starkmap_Vcm[sublevel] = pd.read_csv(filename, index_col=None, header=None, skiprows = 1)
    starkmap_Vm[sublevel] = starkmap_Vcm[sublevel].copy()            #Copy the df
    #starkmap_Vm[sublevel][0] *= 1e2                                  #Convert cm to m
    starkmap_Vm[sublevel][1] *= (constants.h*constants.c)   #Convert energy from wavenumber to joules
    starkmap_Vm[sublevel].columns = [ r'E Field $(V/m)$', 'U $(J)$']    #Label corrected units

In [ ]:
starkmap_Vm['TEST']

#### Initializing the wire and field

In [ ]:
#Defining E(r) and r(E)
def Efield(r):
    """
    Calculate electric field strength at a given radial distance from a wire.
    
    Args:
        r (float): The radial distance from the wire in meters.
        
    Returns:
        float: The electric field strength in volts per meter (V/m) at the specified distance. """
    
    E = (1 / r) * VHere / (math.log(R0 / Rwire))  # Volts per meter
    
    return E

In [ ]:
def r(Efield):
    """
    Calculate the radial distance from a wire corresponding to a given electric field strength.
    
    Args:
        Efield (float): The electric field strength in volts per meter (V/m).
        
    Returns:
        float: The radial distance from the wire in meters that corresponds to the 
        specified electric field strength. """
    
    r = (1 / Efield) * VHere / (math.log(R0 / Rwire))
    
    return r

In [ ]:
#Defining some constants

R0 = 0.15  # R0 = 0.15 m = 15 cm (not super important; imaginary radius at which electric potential is zero)
Rwire = 15e-6  # Radius of the wire = 15 microns (15*10^-6 meters)
VHere = 10  # You can uncomment this line if needed

In [ ]:
zStart = -10**3 * Rwire
zEnd = -Rwire

r_values = np.linspace(0.15 * zStart, zEnd, 100)  # Create an array of z values for the plot
field_values = [Efield(r) for r in r_values]  # Calculate electric field values

plt.figure(figsize=(5, 3))
plt.plot(r_values, field_values)
plt.ylim(0, 0.5 * Efield(zEnd))
plt.xlabel("Position in relation to the Wire(m)")
plt.ylabel("Electric Field (V/m)")
#plt.title("Electric Field vs. Radial Distance")
plt.grid(True)
plt.show()

$\textbf{Note:}$ The negative "Radial Distance" and negative "Electric Field" on the plot represent the geometry of the atoms in relation to the wire. The atoms are below the wire, so if the wire is the origin, the displacement is negative (hence negative values for radial distance). The electric field is pointing outward in the $\hat{r}$-direction, but we can treat this as the $\hat{y}$-direction in relation to the upward-moving atoms. 

#### Calculating Acceleration Vector Field

In [ ]:
# Relating distance and energy for a given field strength

r_of_E = {}
starkmap_r = {}

for sublevel in sublevels:
    starkmap_r[sublevel] = starkmap_Vm[sublevel].copy()  #Create new df
    # Calculate and add column with the radial distance
    starkmap_r[sublevel]['Radial Distance (m)'] = r(starkmap_Vm[sublevel]['E Field $(V/m)$'])  

In [ ]:
starkmap_r['TEST']

In [ ]:
# Creating interpolated function
from scipy.interpolate import interp1d

energy_interpolation = {}
for sublevel in sublevels:
    energy_interpolation[sublevel] = interp1d(starkmap_r[sublevel]['Radial Distance (m)'], starkmap_r[sublevel]['U $(J)$'], kind = 'cubic')

In [ ]:
#Saving U(r)
import pickle

# Define the directory to save the pickle files
output_dir = '/home/anaconda3/Wimberly_PHYS704_F23/U(r)/'

for sublevel in sublevels:
    
    # Define the full path of the output file
    fname = f"U(r)_{sublevel}.pkl"
    output_path = os.path.join(output_dir, fname)

    # Save the interpolation as a pickle file
    with open(output_path, 'wb') as file:
        pickle.dump(energy_interpolation[sublevel], file)

In [ ]:
plt.figure(figsize=(10, 6))
for sublevel in sublevels[25:30]:
    plt.plot(starkmap_r[sublevel]['Radial Distance (m)'][3500::], starkmap_r[sublevel]['U $(J)$'][3500::], 'o', label='Original Data')  # Plot the original data points as dots
    plt.plot(starkmap_r[sublevel]['Radial Distance (m)'][3500::], energy_interpolation[sublevel](starkmap_r[sublevel]['Radial Distance (m)'][3500::]), '-', label='Interpolated Energy')  # Plot the interpolated values as a line
plt.xlabel('Radial Distance (m)')
plt.ylabel('U (J)')
#plt.legend()
plt.grid(True)
#plt.title('Energy vs. Radial Distance')
plt.show()

Here I am selecting the last 500 points from 5 sublevels to have a "zoomed in" look at our interpolation functions. Can plot entire dataset by removing [3500::], however, the peculiarities of the Stark effect are less visible. Can also plot all sublevels by removing [25:30] from the for loop.

In [ ]:
#Attemping to use BSpline as it has built in derivative function
from scipy.interpolate import BSpline

sorted_starkmap_r = {}
energy_interpolation = {}
for sublevel in sublevels:
    sorted_starkmap_r[sublevel] = starkmap_r[sublevel].sort_values(by='Radial Distance (m)')
    energy_interpolation[sublevel] = BSpline(sorted_starkmap_r[sublevel]['Radial Distance (m)'], sorted_starkmap_r[sublevel]['U $(J)$'], k = 1)
    
#K = 1 turns out to produce the best fit

In [ ]:
plt.figure(figsize=(5, 3))
for sublevel in sublevels[25:30]:
    plt.plot(sorted_starkmap_r[sublevel]['Radial Distance (m)'], sorted_starkmap_r[sublevel]['U $(J)$'], 'o', label='Original Data')  # Plot the original data points as dots
    plt.plot(sorted_starkmap_r[sublevel]['Radial Distance (m)'], energy_interpolation[sublevel](sorted_starkmap_r[sublevel]['Radial Distance (m)']), '-', label='Interpolated Energy')  # Plot the interpolated values as a line
plt.xlabel('Radial Distance (m)')
plt.ylabel('U (J)')
#plt.legend()
plt.grid(True)
#plt.title('Energy vs. Radial Distance')
plt.show()

It looks like this is working let's take a look at some slices to double check:

In [ ]:
# Define the data subsets
dataslices = [
    (slice(0, 500), "First 500 Points"),
    (slice(500, 1000), "Points 500 to 1000"),
    (slice(1000, 1500), "Points 1000 to 1500"),
    (slice(1500, 2000), "Points 1500 to 2000"),
    (slice(2000, 2500), "Points 2000 to 2500"),
    (slice(2500, 3000), "Points 2500 to 3000"),
    (slice(3000, 3500), "Points 3000 to 3500"),
    (slice(3500, 4000), "Points 3500 to 4000"),
    ]

# Create subplots for each subset
fig, axes = plt.subplots(4, 2, figsize=(12, 16))
fig.suptitle("Interpolation Plots")

for i, (dataslice, title) in enumerate(dataslices):
    row, col = i // 2, i % 2
    ax = axes[row, col]
    ax.set_title(title)
    for sublevel in sublevels[0:10]:
        ax.plot(
            starkmap_r[sublevel]['Radial Distance (m)'][dataslice],
            starkmap_r[sublevel]['U $(J)$'][dataslice],
            'o',
            label='Original Data'
        )
        ax.plot(
            starkmap_r[sublevel]['Radial Distance (m)'][dataslice],
            energy_interpolation[sublevel](starkmap_r[sublevel]['Radial Distance (m)'][dataslice]),
            '-',
            label='Interpolated Energy'
        )
    ax.set_xlabel('Radial Distance (m)')
    ax.set_ylabel('U (J)')
    ax.grid(True)
    #ax.legend()

# Adjust spacing between subplots
plt.tight_layout()
plt.subplots_adjust(top=0.94)
plt.show()

I think this looks okay... will definitely talk to professor Goodsell. The fit appears to diverge (slightly) further away from the wire while it seems to fit quite well closer. Typically this would feel good however, our particles start further away from the wire. How far? 

In [ ]:
#Solving for grad_U with built in derivative function

grad_U = {}
for sublevel in sublevels: 
    grad_U[sublevel] = energy_interpolation[sublevel].derivative()

#### Outputing $\nabla U$ files:

In [ ]:
#Saving grad U(r)
import pickle

# Define the directory to save the pickle files
output_dir = '/home/anaconda3/Wimberly_PHYS704_F23/grad_U/'

for sublevel in sublevels:
    
    # Define the full path of the output file
    fname = f"grad_U_{sublevel}.pkl"
    output_path = os.path.join(output_dir, fname)

    # Save the interpolation as a pickle file
    with open(output_path, 'wb') as file:
        pickle.dump(grad_U[sublevel], file)

In [ ]:
# #We can't operate on an interpolation so we evaluate it at points to find force

# Force = {}
# for sublevel in sublevels: 
#     Force[sublevel] = -1 * grad_U[sublevel](starkmap_r[sublevel]['Radial Distance (m)'])

In [ ]:
# #Finally we calculate acceleration values
# mRb = (85*10^-3)/(602*10^21)

# acceleration = {}
# for sublevel in sublevels:
#     acceleration[sublevel] = Force[sublevel]/mRb

In [ ]:
# acceleration_map = {}

# for sublevel in sublevels:
#     acceleration_map[sublevel] = starkmap_r[sublevel].copy()

#     # Drop the columns 'E Field $(V/m)$' and 'Energy $(J)$'
#     columns_to_drop = ['E Field $(V/m)$', 'U $(J)$']
#     acceleration_map[sublevel] = acceleration_map[sublevel].drop(columns=columns_to_drop)

#     # Replace the dropped columns with 'Force' and 'Acceleration' values
#     acceleration_map[sublevel]['Force $N$'] = Force[sublevel]  # Replace 'force_values' with your actual force values
#     acceleration_map[sublevel]['Acceleration $m/s$'] = acceleration[sublevel]  # Replace 'acceleration_values' with your actual acceleration values

In [ ]:
# acceleration_map['TEST']

In [ ]:
# #Saving our "acceleration map"

# # Define the directory to save the CSV files
# output_dir = current_directory + '/Acceleration Maps/'

# for sublevel in sublevels:
    
#     #Define the file name
#     fname = f"Accleration_Map_{sublevel}.csv"

#     # Define the full path of the output file
#     output_path = os.path.join(output_dir, fname)

#     # Save the DataFrame as CSV
#     acceleration_map[sublevel].to_csv(output_path, header=True, index=True)